In [ ]:

import requests
import json
import time
import datetime
from urllib.request import urlopen
import requests
import warnings
from copy import copy

import numpy as np
import pandas as pd
import seaborn as sns
import osmnx as ox
import networkx as nx
import geopandas as gpd

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pyproj import CRS
import os
import movingpandas as mpd
from shapely.geometry import Point

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Read the dataframe to find out GPS IDs whose trajectories are known
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Date constants
dat = '01Aug'
dated = '2023-08-01'

# Read and display the trajectory csv
kelowna_gps_01Aug_traj = pd.read_csv(f'/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_11/wkelowna_gps_{dat}_matched_trips_gdf.csv')
kelowna_gps_01Aug_traj

In [ ]:
# List all unique gps_ids
unique_registration_ids = kelowna_gps_01Aug_traj['gps_id'].unique()

print("Number of total unique registration IDs:", len(unique_registration_ids))
print("Unique registration IDs:", unique_registration_ids)

In [ ]:

import matplotlib.pyplot as plt
import contextily as ctx

fpath = '/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim'
wk_neighs = gpd.read_file(f'{fpath}/data/wkelowna/wk_neighbourhoods/wk_neighbourhoods.shp')

# Reproject the GeoDataFrame to EPSG:3857 for compatibility with the basemap
wk_neighs = wk_neighs.to_crs(epsg=3857)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the Lytton square polygon
wk_neighs.plot(ax=ax, color='blue', linewidth=1, alpha=0.5, edgecolor='k')

# Add the basemap using Contextily (OSM)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Show the plot
plt.show()


In [ ]:

import matplotlib.pyplot as plt
import contextily as ctx

fpath = '/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim'
wfn_neighs = gpd.read_file(f'{fpath}/data/wkelowna/wfn_boundary/WFN_-_Boundary_Lines.shp')

# Reproject the GeoDataFrame to EPSG:3857 for compatibility with the basemap
wfn_neighs = wfn_neighs.to_crs(epsg=3857)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the Lytton square polygon
wfn_neighs.plot(ax=ax, color='blue', linewidth=1, alpha=0.5, edgecolor='k')

# Add the basemap using Contextily (OSM)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Show the plot
plt.show()


In [ ]:

# Filter for only IR#9 and IR#10 (The regions in West Kelowna)
wfn_neighs = wfn_neighs[wfn_neighs['CommonName'].isin(['IR#9', 'IR#10'])]

# Drop unnecessary rows
wfn_neighs = wfn_neighs.drop(columns=['OBJECTID', 'ADMIN_AREA', 'Shapearea', 'Shapelen'])
wk_neighs = wk_neighs.drop(columns=['Id', 'Zone'])

# Rename CommonName to Name in wfn_neighs
wfn_neighs.rename(columns={'CommonName': 'Name'}, inplace=True)
wfn_neighs


In [ ]:

# Merge the two dataframes on the Name column
wkelowna_neighs = pd.concat([wfn_neighs, wk_neighs], ignore_index=True)
display(wkelowna_neighs)


In [ ]:

# Read and filter the GPS csv
kelowna_gps_01Aug = pd.read_csv(f'/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/gps_2025_03/kelowna_gps_{dated}.csv')
kelowna_gps_01Aug = kelowna_gps_01Aug[kelowna_gps_01Aug['registrationID'].isin(unique_registration_ids)]
kelowna_gps_01Aug

In [ ]:

import geopandas as gpd
from shapely.geometry import Point

# Convert GPS coordinates to GeoPandas GeoSeries
geometry = [Point(xy) for xy in zip(kelowna_gps_01Aug['longitude'], kelowna_gps_01Aug['latitude'])]
kelowna_gps_01Aug = gpd.GeoDataFrame(kelowna_gps_01Aug, geometry=geometry, crs="EPSG:4326")

# Assuming your first dataframe ('neighborhoods_df') has a CRS. If not, you might need to set it.
# For example, if it's in UTM zone 11N, you would do:
# wkelowna_neighs.crs = "EPSG:32611"

# 2. Transform the GPS GeoDataFrame to the CRS of the neighborhoods GeoDataFrame
kelowna_gps_01Aug = kelowna_gps_01Aug.to_crs(wkelowna_neighs.crs)

# 3. Perform a spatial join
kelowna_gps_01Aug_w_community = gpd.sjoin(kelowna_gps_01Aug, wkelowna_neighs[['Name', 'geometry']], how='left', predicate='within')

# 4. Extract Community Names
kelowna_gps_01Aug_w_community = kelowna_gps_01Aug_w_community.dropna(subset=['Name'])
kelowna_gps_01Aug_w_community

In [ ]:

import pandas as pd
import geopandas as gpd
from shapely import wkt

wk_roads_full_shp = pd.read_csv('/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_12/wk_roads_full_shp.csv')

# Parse the WKT geometry strings into Shapely geometry objects
wk_roads_full_shp['geometry'] = wk_roads_full_shp['geometry'].apply(wkt.loads)

# Create the GeoDataFrame; coordinates are in WGS84 (latitude/longitude)
wk_roads_full_shp = gpd.GeoDataFrame(wk_roads_full_shp, geometry='geometry', crs="EPSG:4326")
wk_roads_full_shp


In [ ]:
# Read trajectory datafile
wk_combined_traj_data = pd.read_csv('/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_11/wk_combined_traj_data.csv')
wk_combined_traj_data

In [ ]:
# Map GPS trajectory points to road segments and create a sequence of roads traversed
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
from tqdm import tqdm

def map_trajectories_to_roads(trajectories_df, road_network_gdf):

    # Create a spatial index for the road network to speed up queries
    road_sindex = road_network_gdf.sindex
    
    # Storage for trip routes
    trip_routes = []
    
    # Process each trajectory
    for _, trajectory in tqdm(trajectories_df.iterrows(), total=len(trajectories_df), desc="Processing trajectories"):
        gps_id = trajectory['gps_id']
        trip_id = trajectory['trip_id']
        day = trajectory['day']
        time_pos_dict = trajectory['time_pos_dict']
        
        # Extract points from the trajectory
        if isinstance(trajectory['geometry'], LineString):
            # Get individual points from the LineString
            traj_points = [Point(coord) for coord in trajectory['geometry'].coords]
        else:
            # Skip if geometry is not a LineString
            continue
        
        # Create ordered lists to maintain sequence
        road_ids_sequence = []
        road_names_sequence = []
        
        # Match each point to the nearest road segment
        for point in traj_points:
            # Find potential road candidates using spatial index
            potential_matches_idx = list(road_sindex.intersection(point.bounds))
            potential_matches = road_network_gdf.iloc[potential_matches_idx]
            
            # If no potential matches, continue to next point
            if len(potential_matches) == 0:
                continue
            
            # Find the closest road segment
            distances = potential_matches.distance(point)
            nearest_road_idx = distances.idxmin()
            nearest_road = road_network_gdf.loc[nearest_road_idx]
            
            # Only consider roads within a certain threshold (e.g., 30 meters)
            min_distance = distances.min()
            if min_distance <= 30:  # 30 meters threshold
                road_id = nearest_road['STR_ID']
                road_name = nearest_road['NAME']
                
                # Add to sequence (we'll remove duplicates later)
                road_ids_sequence.append(road_id)
                road_names_sequence.append(road_name)
        
        # Remove consecutive duplicates to get the actual sequence of roads
        # This preserves cases where a vehicle returns to a road after taking other roads
        unique_road_ids = []
        unique_road_names = []
        
        for i, (road_id, road_name) in enumerate(zip(road_ids_sequence, road_names_sequence)):
            # Add to unique list if it's the first item or different from the previous one
            if i == 0 or road_id != road_ids_sequence[i-1]:
                unique_road_ids.append(road_id)
                unique_road_names.append(road_name)
        
        # Store result for this trip
        if unique_road_ids:  # Only store if we found matching roads
            trip_routes.append({
                'day': day,
                'gps_id': gps_id,
                'trip_id': trip_id,
                'road_ids': unique_road_ids,
                'road_names': unique_road_names,
                'time_pos_dict': time_pos_dict
            })
    
    # Create DataFrame from results
    trip_routes_df = pd.DataFrame(trip_routes)
    
    return trip_routes_df

In [ ]:

# Make sure both GeoDataFrames use the same CRS (projected)
wk_roads_full_shp = wk_roads_full_shp.to_crs(epsg=26911)       # UTM zone for Kelowna

# Only apply wkt.loads to strings
wk_combined_traj_data['geometry'] = wk_combined_traj_data['geometry'].apply(
    lambda x: wkt.loads(x) if isinstance(x, str) else x
)

# Convert to GeoDataFrame
wk_combined_traj_data = gpd.GeoDataFrame(wk_combined_traj_data, geometry='geometry', crs='EPSG:4326')  # assuming it's in WGS84
wk_combined_traj_data = wk_combined_traj_data.to_crs(epsg=26911)  # UTM zone 11N (Kelowna)

travelled_roads_1 = map_trajectories_to_roads(wk_combined_traj_data, wk_roads_full_shp)
travelled_roads_1

In [ ]:
# Save the dataframe as csv
csv_filepath = 'travelled_roads.csv'

# Save as new csv or append to existing csv
travelled_roads_1.to_csv(csv_filepath)


<H3>East Kelowna</H3>


In [ ]:

import pandas as pd
import geopandas as gpd
from shapely import wkt

# Date constants
dat = '01Aug'
dated = '2023-08-01'

# Read and display the trajectory csv
kelowna_gps_01Aug_traj = pd.read_csv(f'/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_11/EKelowna/ekelowna_gps_{dat}_matched_trips_gdf.csv')
kelowna_gps_01Aug_traj

In [ ]:

import matplotlib.pyplot as plt
import contextily as ctx

fpath = '/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim'
ek_neighs = gpd.read_file(f'{fpath}/data/wkelowna/ek_neighbourhoods/Kelowna_Neighborhood.shp')

# Reproject the GeoDataFrame to EPSG:3857 for compatibility with the basemap
ek_neighs = ek_neighs.to_crs(epsg=3857)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the Lytton square polygon
ek_neighs.plot(ax=ax, color='blue', linewidth=1, alpha=0.5, edgecolor='k')

# Add the basemap using Contextily (OSM)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Show the plot
plt.show()


In [ ]:
# Import road network map of West Kelowna
import pandas as pd
import geopandas as gpd
from shapely import wkt

### wk_roads_full_shp = pd.read_csv('/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_12/wk_roads_full_shp.csv')
ek_roads_full_shp = pd.read_csv('/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_12/ek_georoads_shp.csv')

# Parse the WKT geometry strings into Shapely geometry objects
ek_roads_full_shp['geometry'] = ek_roads_full_shp['geometry'].apply(wkt.loads)

# Create the GeoDataFrame; coordinates are in WGS84 (latitude/longitude)
ek_roads_full_shp = gpd.GeoDataFrame(ek_roads_full_shp, geometry='geometry', crs="EPSG:4326")
ek_roads_full_shp


In [ ]:
ek_combined_traj_data = pd.read_csv('/media/tim/114249c0-882b-4e87-8cf4-b6d65df30eb21/tim/data/outputs/traj_2024_11/EKelowna/ek_combined_traj_data.csv')
ek_combined_traj_data

In [ ]:
# Map GPS trajectory points to road segments and create a sequence of roads traversed
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
from tqdm import tqdm

def map_trajectories_to_roads(trajectories_df, road_network_gdf):

    # Create a spatial index for the road network to speed up queries
    road_sindex = road_network_gdf.sindex
    
    # Storage for trip routes
    trip_routes = []
    
    # Process each trajectory
    for _, trajectory in tqdm(trajectories_df.iterrows(), total=len(trajectories_df), desc="Processing trajectories"):
        gps_id = trajectory['gps_id']
        trip_id = trajectory['trip_id']
        day = trajectory['day']
        time_pos_dict = trajectory['time_pos_dict']
        
        # Extract points from the trajectory
        if isinstance(trajectory['geometry'], LineString):
            # Get individual points from the LineString
            traj_points = [Point(coord) for coord in trajectory['geometry'].coords]
        else:
            # Skip if geometry is not a LineString
            continue
        
        # Create ordered lists to maintain sequence
        road_ids_sequence = []
        road_names_sequence = []
        
        # Match each point to the nearest road segment
        for point in traj_points:
            # Find potential road candidates using spatial index
            potential_matches_idx = list(road_sindex.intersection(point.bounds))
            potential_matches = road_network_gdf.iloc[potential_matches_idx]
            
            # If no potential matches, continue to next point
            if len(potential_matches) == 0:
                continue
            
            # Find the closest road segment
            distances = potential_matches.distance(point)
            nearest_road_idx = distances.idxmin()
            nearest_road = road_network_gdf.loc[nearest_road_idx]
            
            # Only consider roads within a certain threshold (e.g., 30 meters)
            min_distance = distances.min()
            if min_distance <= 30:  # 30 meters threshold
                road_id = nearest_road['road_id']
                road_name = nearest_road['roadmst_na']
                
                # Add to sequence (we'll remove duplicates later)
                road_ids_sequence.append(road_id)
                road_names_sequence.append(road_name)
        
        # Remove consecutive duplicates to get the actual sequence of roads
        # This preserves cases where a vehicle returns to a road after taking other roads
        unique_road_ids = []
        unique_road_names = []
        
        for i, (road_id, road_name) in enumerate(zip(road_ids_sequence, road_names_sequence)):
            # Add to unique list if it's the first item or different from the previous one
            if i == 0 or road_id != road_ids_sequence[i-1]:
                unique_road_ids.append(road_id)
                unique_road_names.append(road_name)
        
        # Store result for this trip
        if unique_road_ids:  # Only store if we found matching roads
            trip_routes.append({
                'day': day,
                'gps_id': gps_id,
                'trip_id': trip_id,
                'road_ids': unique_road_ids,
                'road_names': unique_road_names,
                'time_pos_dict': time_pos_dict
            })
    
    # Create DataFrame from results
    trip_routes_df = pd.DataFrame(trip_routes)
    
    return trip_routes_df

In [ ]:
# Run the above function
ek_roads_full_shp = ek_roads_full_shp.to_crs(epsg=26911)       # UTM zone for Kelowna

# Only apply wkt.loads to strings
ek_combined_traj_data['geometry'] = ek_combined_traj_data['geometry'].apply(
    lambda x: wkt.loads(x) if isinstance(x, str) else x
)

# Convert to GeoDataFrame
ek_combined_traj_data = gpd.GeoDataFrame(ek_combined_traj_data, geometry='geometry', crs='EPSG:4326')  # assuming it's in WGS84
ek_combined_traj_data = ek_combined_traj_data.to_crs(epsg=26911)  # UTM zone 11N (Kelowna)

travelled_roads_1 = map_trajectories_to_roads(ek_combined_traj_data, ek_roads_full_shp)
travelled_roads_1